<div style="display: flex; align-items: center;">
    <div>
        <h2>Actividad WhitePapers</h2>
        <h2>Miguel López</h2>
        <h2>ID 1001014378</h2>
    </div>
    <img src="https://yt3.ggpht.com/-10IUL9wra6k/AAAAAAAAAAI/AAAAAAAAAAA/UOBLu1uYOOE/s900-c-k-no/photo.jpg" alt="Image description" width="200" style="margin-left: 20px;"/>
</div>

# Una pequeña introducción a los transformers. LoRa, QLoRa y sus efectos en el fine-tuning de Large Language Models.

# 1. Introducción
Gracias al gran éxito de los Transformers, en especial de chatGPT por OpenAI y el gran interés por la industria en adoptar estas nuevas tecnologías. Ha generado la necesidad de modelos más exactos para diferentes casos de uso a nivel profesional. Para ello tradicionalmente se realizaba un reentrenamiento supervisado (con datos etiquetados) completo de los modelos. El cual podía dar a luz problemas como la pérdida de memoria del modelo para otras tareas [1] o principalmente el gran gasto computacional que implica entrenar un modelo por ejemplo GPT 3 con 175 billones de parámetros [2]. Además a diferencia de algunas técnicas de Deep learning no se puede aplicar simplemente un entrenamiento a las capas superficiales del modelo ya que por la naturaleza de los Transformers, reentrenar su ventana de atención, en otras palabras lo que le permite a estos modelos tener una coherencia de ideas con un texto dado [3]. Para dar solución a todo esto en 2021 se presentó LoRa o Low-Rank Adaptation of Large Language Models [2], donde a través de una adición de una pequeña matriz en capas especificas del modelo se podría reentrenar a un costo computacional menor, sin perder la mayoría de sus cualidades generales como Large Lenaguage Models (LLM) y con la capacidad de respuestas más precisas dado casos establecidos.


Pero pese a esto, en términos de optimización de memoria se puede combinar con técnicas como cuantización para bajar el grado de precisión de los pesos flotantes de los modelos de 32 bits a 16, 8 o 4 bits y luego realizando un fine-tuning con LoRa, conocido como QLoRA [4]. Perdiendo un poco de precisión en el momento de sus respuestas, pero dejando las puertas abiertas a aplicaciones en por ejemplo teléfonos celulares o sistemas embebidos para su uso como productos funcionales.  


# 2 Funcionamiento de Transformers y sus implicaciones.

Los transfomers parten de la idea del como mantener la atención de un modelo a lo largo de sus neuronas sin perder ese foco de atención entre ellas. Usando la arquitectura de la forma:


<div style="text-align: center;">
    <img src="./images/transformers.png" alt="Fig 1. Arquitectura de Transformers [3]">
    <p><em>Fig 1. Arquitectura de Transformers [3]</em></p>
</div>

Donde inicialmente nos interesa la etapa de encoding, donde las palabras son traducidas a un espacio vectorial dependiendo de su similitud semántica. Como se puede observar a continuación:

<div style="text-align: center;">
    <img src="./images/3d_wordvec_3.png" alt="Fig 2. Demostración de un embedings en un espacio 3D [5]" width="50%">
    <p><em>Fig 2. Demostración de un embedings en un espacio 3D [5]</em></p>
</div>

De esta manera pudiendo operar matricialmente las palabras (en forma de vectores) con la arquitectura mostrada en la Fig 1.

Anteriormente usando redes neuronales recurrentes (RNN) las cuales después de cada capa iban perdiendo la atención al texto ingresado debido a su naturaleza secuencial, donde en el entrenamiento se puede dar el fenómeno del desvanecimiento de gradientes, el cual implica que los gradientes a medida que pasan por la backpropagation se vuelven 0 o un valor cercano a este impidiendo al modelo aprender nueva información [3] además que la complejidad para volver a esa información requiere un tamaño menor $O(1)$ de Transformers comparado con el $O(n)$ de RNN [3]. Esto se soluciona mediante capas de atención las cuales son dadas por una serie de matrices a las cuales se les aplica una función softmax de la forma:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$


Recordando que la función softmax  es [6]:

$$
\text{softmax}(x_i) = \frac{\exp(x_i)}{\sum_{j} \exp(x_j)}
$$

In [3]:
# Iniciando con numpy aunque este proceso se hace por medio de tensores

import numpy as np

d_k = 4  # Dimensión de las llaves
sequence_length = 3  # Número de elementos en la secuencia
dimension = 4  # Dimensión de cada vector en Q, K, y V

# Generamos valores aleatorios para Q, K, y V
np.random.seed(23)  # Semilla para reproducibilidad
Q = np.random.rand(sequence_length, dimension)
K = np.random.rand(sequence_length, dimension)
V = np.random.rand(sequence_length, dimension)

# Calculamos la atención
# QK^T
QK_T = np.dot(Q, K.T)

# Dividimos por la raíz cuadrada de d_k
QK_T_scaled = QK_T / np.sqrt(d_k)

# Aplicamos la función softmax
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

attention_weights = softmax(QK_T_scaled)

# Calculamos la salida de la atención
output = np.dot(attention_weights, V)

print("Q:")
print(Q)
print("\nK:")
print(K)
print("\nV:")
print(V)
print("\nAtención:")
print(attention_weights)
print("\nSalida de la atención:")
print(output)

Q:
[[0.51729788 0.9469626  0.76545976 0.28239584]
 [0.22104536 0.68622209 0.1671392  0.39244247]
 [0.61805235 0.41193009 0.00246488 0.88403218]]

K:
[[8.84947538e-01 3.00409689e-01 5.89581865e-01 9.78426916e-01]
 [8.45093822e-01 6.50754391e-02 2.94744465e-01 2.87934441e-01]
 [8.22466339e-01 6.26183038e-01 1.10477714e-01 5.28811169e-04]]

V:
[[0.94216623 0.14150076 0.42159653 0.34648944]
 [0.86978508 0.42860181 0.82875148 0.71785184]
 [0.11922669 0.59638417 0.1297563  0.07753409]]

Atención:
[[0.39217151 0.28135736 0.32647113]
 [0.37549183 0.2938171  0.33069108]
 [0.42128813 0.29205974 0.28665214]]

Salida de la atención:
[[0.65313526 0.37078505 0.44087516 0.36316883]
 [0.64876065 0.37628184 0.44471666 0.36666093]
 [0.68512924 0.35574472 0.45685347 0.37785282]]


# Tal vez agregar lo de las cabezas?

En este ejemplo usamos 3 dimensiones, aunque en realidad la entrada de las matrices de atención son mayores, por ejemplo si quisiéramos reentrenar un modelo como GPT 3 el cual cuenta con 16.385 dimensiones [7] y 175 billones de parámetros [2]. Se incrementa el gasto computacional para cada iteración. Por ende el uso de menos parámetros especializados para ajustar un modelo es necesario y una de estas soluciones es LoRa

## 3.LoRa [2]
Este permite un ajuste eficiente, mediante la descomposición de bajo rango de las matrices de atención. Permitiendo un mejor ajuste con pocos parámetros. Esto quiere decir que si partimos de el cómo funciona una LLM de forma general tenemos que siendo $W_0$ las dimensiones de los pesos iniciales de un modelo entrenado su ajuste esta dado por un $\Delta W$ para que este de una salida de predicción $h$ de la forma:
$$
h = W_0 x + \Delta W x
$$

Supongamos así que existe una descomposición para $\Delta W$ donde $B \in \mathbb{R}^{d \times r}$ y $A \in \mathbb{R}^{r \times k}$, donde r es el rank y d las dimensiones de la matriz del modelo. Así si $\nabla(W) x = BA x$ tenemos que:
$$
h = W_0 x + BA x
$$

De una manera gráfica se puede expresar mediante la comparación de:

<div style="text-align: center;">
    <img src="./images/reaparam.png" alt="Fig 3. Reparametrización en la que se basa LoRa [2]" width="50%">
    <p><em>Fig 2. Demostración de un embedings en un espacio 3D [5]</em></p>
</div>


Así los pesos de $A$ y $B$ son cambiados en el fine-tuning del modelo.

In [179]:
#De una manera practica se tiene:

#Supongamos un valor h

h = np.random.rand(dimension)

#Donde los pesos iniciales son aleatorios
W = np.random.rand(dimension, dimension)

#Establecemos un X de entrada 

X = np.random.rand(dimension, 1)

def normal_tuning(W, X, h):
    #Calculamos el producto punto de W y X
    WX = np.dot(W, X)
    #Generamos la matriz delta_W en ceros
    delta_W = np.zeros_like(W)
    #Ajustamos los pesos de delta_W a base de h
    for i in range(W.shape[0]):
        delta_W[i], _, _, _ = np.linalg.lstsq(X.T, h[i] - WX[i], rcond=None)
    return delta_W

delta_W = normal_tuning(W, X, h)

#Solución de h
normal_h = np.dot(W, X) + np.dot(delta_W, X)

print("Valor de h:")
print(h)

print("\nAjuste de pesos normal:")
print(normal_h.T)

print("\nCon una matriz delta_W:")
print(delta_W)

#De la misma manera podemos ajustar los pesos de A y B, en este caso para ser prácticos solo ajustaremos
#Los pesos de B al ser todos 0

#Creamos la clase de LoRa
class LoRa_tuning:
    def __init__(self, A, h, W, X,r):
        #Inicializamos las variables
        self.A = A
        self.h = h
        self.W = W
        self.X = X
        self.B = np.zeros((W.shape[0], r))
        self.d = W.shape[0]
    def __call__(self):
        #Calculamos los productos punto de A y X y W y X
        Ax = np.dot(self.A, self.X)
        Wx = np.dot(self.W, self.X)
        #Ajustamos los pesos de B a base de h
        for i in range(self.B.shape[0]):
            self.B[i], _, _, _ = np.linalg.lstsq(Ax.T, h[i] - Wx[i], rcond=None)
        return self.B
    def solution(self):
        # Calculamos la solución
        return np.dot(self.W, self.X) + np.dot(np.dot(self.B, self.A), self.X)
    def is_equal(self):
        #Confirmamos que los tamaños de W y A*B sean iguales
        return self.W.shape == np.dot(self.B, self.A).shape
    def a_dot_b(self):
        #Calculamos A*B
        return np.dot(self.B, self.A)

r = 2
A = np.random.rand(r, dimension)


LoRa = LoRa_tuning(A, h, W, X,r)
B = LoRa()

#Confirmamos que los tamaños de W y A*B sean iguales
print("\nTamaños de W y A*B iguales:", LoRa.is_equal())

print("\nAjuste de pesos con A y B:")
print(LoRa.solution().T)

print("\nCon A:")
print(LoRa.A)

print("\nCon B:")
print(B)




Valor de h:
[0.27102661 0.70994164 0.07783213 0.14330376]

Ajuste de pesos normal:
[[0.27102661 0.70994164 0.07783213 0.14330376]]

Con una matriz delta_W:
[[-0.05888941 -0.33965381 -0.0782992  -0.15470414]
 [ 0.08399062  0.48442892  0.1116737   0.22064573]
 [-0.10498797 -0.60553444 -0.13959173 -0.27580638]
 [-0.13069238 -0.75378857 -0.17376823 -0.34333258]]

Tamaños de W y A*B iguales: True

Ajuste de pesos con A y B:
[[0.27102661 0.70994164 0.07783213 0.14330376]]

Con A:
[[0.15353197 0.89234285 0.08409697 0.20951407]
 [0.28986374 0.15814757 0.37719791 0.25208843]]

Con B:
[[-0.36637554 -0.14536921]
 [ 0.52254061  0.20733185]
 [-0.65317391 -0.25916408]
 [-0.81309171 -0.32261571]]


Obteniendo el mismo resultado de ajuste, con menos parámetros que calcular, en nuestro ejemplo solo calculando 8 valores de la matriz $B$ a diferencia de los 16 del $\Delta W$.